In [8]:
import numpy as np
import numpy_financial as npf
import pandas as pd
import pprint
%matplotlib inline
import matplotlib.pyplot as plt
import yaml

In [9]:
class LCOEModel:
    """
    Class to reproduce Lazard's LCOE results. Monetary amounts are in millions.
    https://www.lazard.com/media/450784/lazards-levelized-cost-of-energy-version-120-vfinal.pdf
    Construction time does not count because it is included in the capital cost.
    """
    def __init__(self, capacity_mw, capacity_factor, fixed_oam_per_kw, capital_cost_per_kw, lifetime,
                 variable_oam_per_mwh=0, fuel_cost_per_mbtu=0, heat_rate_btu_per_kwh=0, oam_esc_rate=0.0225, debt=0.6,
                 debt_cost=0.08, equity=0.4, equity_cost=0.12, tax_rate=0.4, macrs_schedule=5, tol=1e-3, irr_years=20):
        
        self.capacity_mw = capacity_mw
        self.capacity_factor = capacity_factor
        self.fuel_cost_per_mwh = fuel_cost_per_mbtu * heat_rate_btu_per_kwh / 1e9  #/ 1000 / 1e6
        self.fixed_oam_per_kw = fixed_oam_per_kw
        self.variable_oam_per_mwh = variable_oam_per_mwh
        self.capital_cost_per_kw = capital_cost_per_kw
        self.lifetime = lifetime
        
        self.oam_esc_rate = oam_esc_rate
        self.debt = debt
        self.debt_cost = debt_cost
        self.equity = equity
        self.equity_cost = equity_cost
        self.tax_rate = tax_rate
        self.macrs_schedule = macrs_schedule 

        self.capex = capital_cost_per_kw * capacity_mw / 1000  # x1000/1e6
        self.loan = debt * self.capex
        
        # values from https://github.com/elasanchez/depreciation-engineering-econ-scripts
        # if I understand well, they are set by law
        self.macrs_dic = {
            3: [0.3333, 0.4445, 0.1481, 0.0741],
            5: [0.2, 0.32, 0.1920, 0.1152, 0.1152, 0.0576],
            7: [0.1429,0.2449, 0.1749, 0.1249, 0.0893, 0.0892, 0.0893, 0.0446],
            10: [0.1, 0.18, 0.144, 0.1152, 0.0922, 0.0737, 0.0655, 0.0655, 0.0656, 0.0655, 0.0328],
            15: [0.05, 0.095, 0.0855, 0.077, 0.0693, 0.0623, 0.059, 0.059, 0.0591, 0.059, 0.0591, 0.059, 0.0591,\
                 0.059, 0.0591, 0.0295],
            20: [0.0375, 0.07219, 0.06677, 0.06177, 0.05713, 0.05285, 0.04888, 0.04522, 0.04462, 0.04461, 0.04462,\
                 0.04461, 0.04462, 0.04461, 0.04462, 0.04461, 0.04462, 0.04461, 0.04462, 0.04461, 0.02231]
        }
        
        self.tol = tol
        self.irr_years = irr_years
        self.build()
    
    def build(self, lcoe=None):
        years = np.arange(1, self.lifetime + 1)
        df = pd.DataFrame(index=years)
                
        # we start by computing everything we can compute without assuming a LCOE
        df['capacity_mw'] = self.capacity_mw
        df['capacity_factor'] = self.capacity_factor
        df['generation_mwh'] = df.capacity_mw * df.capacity_factor * 24 * 365.25
        
        df['total_fuel_cost'] = self.fuel_cost_per_mwh * df.generation_mwh
        fixed_oam = self.fixed_oam_per_kw * df.capacity_mw / 1000
        variable_oam = self.variable_oam_per_mwh * df.generation_mwh / 1e6
        total_oam_start = fixed_oam + variable_oam
        df['total_oam'] = total_oam_start * (1 + self.oam_esc_rate)**(years - 1)
        df['total_operating_cost'] = df.total_fuel_cost + df.total_oam
        
        df['debt_outstanding'] = self.loan
        df['debt_interest_expense'] = npf.ipmt(self.debt_cost, df.index, self.lifetime, self.loan)
        df['debt_principal_payment'] = npf.ppmt(self.debt_cost, df.index, self.lifetime, self.loan)          
        df['levelized_debt_service'] = df.debt_interest_expense + df.debt_principal_payment
        df.loc[2:, 'debt_outstanding'] += df['debt_principal_payment'].cumsum().values[:-1]
        
        depreciation = np.array(self.macrs_dic[self.macrs_schedule]) * self.capex
        padding = [0] * (self.lifetime - self.macrs_schedule - 1) if self.lifetime > self.macrs_schedule else []
        df['depreciation'] = list(-depreciation[:self.lifetime]) + padding
        
        # if a lcoe value is given, we can compute the rest
        if lcoe:
            df['lcoe'] = lcoe
            df['total_revenues'] = df.generation_mwh * df.lcoe / 1e6

            df['ebitda'] = df.total_revenues - df.total_operating_cost
            df['taxable_income'] = df.ebitda + df.depreciation + df.debt_interest_expense
            df['tax_benefit'] = - df.taxable_income * self.tax_rate
            df['net_equity_cash_flow'] = df.ebitda + df.levelized_debt_service + df.tax_benefit
            df.loc[0] = 0
            df.loc[0, 'net_equity_cash_flow'] = - self.capex * self.equity
            df = df.sort_index()
            self.irr_equity_investors = npf.irr(df.loc[:self.irr_years+1, 'net_equity_cash_flow'])
            
        self.df = df
        return self
        
    def find_lcoe(self, debug=False):
        """Optimization by dichotomy. IRR is monotonously increasing with LCOE, which makes it easy."""
        lb, ub = 1, 500
        i = 0
        while True:
            lcoe = (lb + ub) / 2
            self.build(lcoe)
            irr = self.irr_equity_investors
            if debug:
                print(round(lb, 5), round(ub, 5), end=' ')
                
            if abs(irr - self.equity_cost) < self.tol:
                self.lcoe = lcoe
                break
            elif irr < self.equity_cost or np.isnan(irr):
                # IRR is NaN if too negative
                lb = lcoe
            elif irr > self.equity_cost:
                ub = lcoe
                
            if debug:
                print(round(irr, 5), end=' | ')
                
            if i == 50:
                print('Project is not viable')
                self.lcoe = '>500'
                break
                
            i += 1
                
        return self.lcoe
    
    def scan_irrs(self):
        irrs = []
        lcoes =  np.arange(1, 300, 5)
        for lcoe in lcoes:
            self.build(lcoe)
            irrs.append(self.irr_equity_investors)
        return lcoes, irrs
    
    def __repr__(self):
        return pprint.pformat({k: v for k, v in vars(self).items() if k not in ['macrs_dic', 'years', 'df']})

In [18]:
with open('power_plants_v13.yml') as f:
    power = yaml.safe_load(f)
    
with open('lazard_estimates_v13.yml') as f:
    lazard = yaml.safe_load(f)

In [19]:
for k, v in power.items():
    print(' '.join(k.split('_')).capitalize())
    model = LCOEModel(**v['low'])
    print('\tMine:\t', int(model.find_lcoe()), end= ' - ')
    model = LCOEModel(**v['high'])
    print(int(model.find_lcoe()))
    print('\tLazard:\t {} - {}'.format(lazard[k]['low'], lazard[k]['high']))

Pv utility thin film
	Mine:	 35 - 41
	Lazard:	 32 - 42
Nuclear
	Mine:	 120 - 197
	Lazard:	 118 - 192
Pv utility crystalline
	Mine:	 37 - 46
	Lazard:	 36 - 44
Gas peaking
	Mine:	 151 - 203
	Lazard:	 150 - 199
Geothermal
	Mine:	 72 - 117
	Lazard:	 69 - 112
Pv community
	Mine:	 66 - 154
	Lazard:	 64 - 148
Pv rooftop cai
	Mine:	 78 - 162
	Lazard:	 75 - 154
Wind offshore
	Mine:	 67 - 121
	Lazard:	 64 - 115
Gas combined cycle
	Mine:	 41 - 66
	Lazard:	 44 - 68
Solar thermal
	Mine:	 134 - 164
	Lazard:	 126 - 156
Coal
	Mine:	 66 - 154
	Lazard:	 66 - 152
Wind onshore
	Mine:	 29 - 57
	Lazard:	 28 - 54
Pv rooftop residential
	Mine:	 158 - 255
	Lazard:	 151 - 242


All but gas are very close to Lazard's results.
### My estimates

In [20]:
print('Onshore wind - Real capacity factor')
a = power['wind_onshore']
d = a['low'].copy()
d['capacity_factor'] = 0.4
model = LCOEModel(**d)
print('Low estimate:', model.find_lcoe())
d = a['high'].copy()
d['capacity_factor'] = 0.25
model = LCOEModel(**d)
print('High estimate:', model.find_lcoe())

Onshore wind - Real capacity factor
Low estimate: 40.4716796875
High estimate: 87.2529296875


In [21]:
print('PV utility scale - Real capacity factor')
a = power['pv_utility_crystalline']
d = a['low'].copy()
d['capacity_factor'] = 0.25
model = LCOEModel(**d)
print('Low estimate:', model.find_lcoe())
d = a['high'].copy()
d['capacity_factor'] = 0.12
model = LCOEModel(**d)
print('High estimate:', model.find_lcoe())

PV utility scale - Real capacity factor
Low estimate: 47.78125
High estimate: 80.4306640625


In [22]:
print('Nuclear - Adapted debt structure')
a = power['nuclear']
d = a['low'].copy()
d['debt'] = 0.8
d['debt_cost'] = 0.02
d['equity'] = 0.2
#d['equity_cost'] = 0.08
d['lifetime'] = 60
model = LCOEModel(**d)
print('Low estimate:', model.find_lcoe())
d = a['high'].copy()
d['debt'] = 0.8
d['debt_cost'] = 0.02
d['equity'] = 0.2
#d['equity_cost'] = 0.08
d['lifetime'] = 60
model = LCOEModel(**d)
print('High estimate:', model.find_lcoe())

Nuclear - Adapted debt structure
Low estimate: 61.42578125
High estimate: 91.638671875


That's much better. Let's give the same debt cost to solar and wind to be fair. I tried with the same financing structure but it makes them unviable.

In [23]:
print('Onshore wind - Real capacity factor and low debt cost')
a = power['wind_onshore']
d = a['low'].copy()
d['capacity_factor'] = 0.4
d['debt_cost'] = 0.02
model = LCOEModel(**d)
print('Low estimate:', model.find_lcoe())
d = a['high'].copy()
d['capacity_factor'] = 0.25
d['debt_cost'] = 0.02
model = LCOEModel(**d)
print('High estimate:', model.find_lcoe())

Onshore wind - Real capacity factor and low debt cost
Low estimate: 33.89306640625
High estimate: 73.12109375


In [24]:
print('PV utility scale - Real capacity factor and low debt cost')
a = power['pv_utility_crystalline']
d = a['low'].copy()
d['capacity_factor'] = 0.25
d['debt_cost'] = 0.02
model = LCOEModel(**d)
print('Low estimate:', model.find_lcoe())
d = a['high'].copy()
d['capacity_factor'] = 0.12
d['debt_cost'] = 0.02
model = LCOEModel(**d)
print('High estimate:', model.find_lcoe())

PV utility scale - Real capacity factor and low debt cost
Low estimate: 37.060546875
High estimate: 62.400390625


It's cheaper but the difference is not nearly as significant as with nuclear.